### Import

In [1]:
# import libraries
import pandas as pd
import sqlite3
from tqdm.notebook import tqdm
import re

### Columns to keep

In [7]:
nppes = pd.read_csv('data/NPPES_Data_Dissemination_February_2022/npidata_pfile_20050523-20220213_FileHeader.csv')
nppes.columns

Index(['NPI', 'Entity Type Code', 'Replacement NPI',
       'Employer Identification Number (EIN)',
       'Provider Organization Name (Legal Business Name)',
       'Provider Last Name (Legal Name)', 'Provider First Name',
       'Provider Middle Name', 'Provider Name Prefix Text',
       'Provider Name Suffix Text',
       ...
       'Healthcare Provider Taxonomy Group_7',
       'Healthcare Provider Taxonomy Group_8',
       'Healthcare Provider Taxonomy Group_9',
       'Healthcare Provider Taxonomy Group_10',
       'Healthcare Provider Taxonomy Group_11',
       'Healthcare Provider Taxonomy Group_12',
       'Healthcare Provider Taxonomy Group_13',
       'Healthcare Provider Taxonomy Group_14',
       'Healthcare Provider Taxonomy Group_15', 'Certification Date'],
      dtype='object', length=330)

In [8]:
# columns to keep
referral_from_cols = ['referral_id', 'from_npi']
referral_to_cols = ['referral_id', 'to_npi']
referrals_cols = ['referral_id',
             'patient_count', 
             'transaction_count', 
             'average_day_wait',
             'std_day_wait']
profile_cols = ['NPI',
               'Entity Type Code',
                'Provider Organization Name (Legal Business Name)',
                'Provider Last Name (Legal Name)',
                'Provider First Name',
                'Provider Middle Name', 
                'Provider Name Prefix Text',
                'Provider Name Suffix Text',
                'Provider Credential Text',
                'Provider First Line Business Practice Location Address',
                'Provider Second Line Business Practice Location Address',
                'Provider Business Practice Location Address City Name',
                'Provider Business Practice Location Address State Name',
                'Provider Business Practice Location Address Postal Code'
               ]
taxonomy_code_cols = [col for col in nppes.columns if 'Healthcare Provider Taxonomy Code' in col ]
taxonomy_switch_cols = [col for col in nppes.columns if 'Healthcare Provider Primary Taxonomy Switch' in col]
taxonomy_cols = ['Code', 'Classification']
zip_cbsa_cols = ['zip', 'cbsa', 'usps_zip_pref_city','usps_zip_pref_state']

### Write hop teaming tables to sqlite3 db

In [53]:
# create a reference_id and write hop_teaming_2018 csv to three tables in hop_teaming db
db = sqlite3.connect('data/hopteam.sqlite')
for chunk in tqdm(pd.read_csv('data/DocGraph_Hop_Teaming_2018_Commercial/DocGraph_Hop_Teaming_2018.csv', chunksize = 10000)):
    # create a unique referral_id for each referal
    chunk = chunk.reset_index().rename(columns = {'index':'referral_id'}) 
    # select columns to keep and write tables to db
    chunk.loc[:,referral_from_cols].to_sql('referral_from', db, if_exists = 'append', index = False) 
    chunk.loc[:,referral_to_cols].to_sql('referral_to', db, if_exists = 'append', index = False)
    chunk.loc[:,referrals_cols].to_sql('referrals', db, if_exists = 'append', index = False)
    db.close()    

0it [00:00, ?it/s]

### Write NPPES table to sqlite3 db

In [ ]:
%%time
# get taxonomy code for each npi and select 15 columns NPPES table to write into the db
db = sqlite3.connect('data/hopteam.sqlite')
for chunk in tqdm(pd.read_csv('data/NPPES_Data_Dissemination_February_2022/npidata_pfile_20050523-20220213.csv', chunksize=100000)):
    # drop NaN for NPI and Zip Code columns
    chunk.dropna(subset= ['NPI','Provider Business Practice Location Address Postal Code'], inplace = True)
    # melt columns of taxonomy codes
    taxonomy_code = chunk.loc[:, ['NPI'] +taxonomy_code_cols]
    taxonomy_code = pd.melt(taxonomy_code, id_vars = 'NPI', value_vars = taxonomy_code_cols)
    taxonomy_code['match_num']=taxonomy_code['variable'].str.extractall('(\d+)').unstack()
    taxonomy_code = taxonomy_code.rename(columns={'value':'taxonomy_code'})
    # melt columns of taxonomy switchs
    taxonomy_switch = chunk.loc[:, ['NPI']+taxonomy_switch_cols]
    taxonomy_switch = pd.melt(taxonomy_switch, id_vars = 'NPI', value_vars = taxonomy_switch_cols)
    taxonomy_switch = taxonomy_switch[taxonomy_switch['value']=='Y']
    taxonomy_switch['match_num'] = taxonomy_switch['variable'].str.extractall('(\d+)').unstack()
    # inner join the taxonomy codes table with taxonomy switch table and other profile columns
    chunk = (pd.merge(taxonomy_code, taxonomy_switch, how = 'inner', on = ['NPI', 'match_num']).
     drop(columns=['variable_x','variable_y', 'match_num','value'])).merge(chunk.loc[:,profile_cols], on ='NPI')
    # extract 5-digit zip codes and assign them to the Business Practice Zip Code column
    correct_zip_code = []
    for zip_code in chunk['Provider Business Practice Location Address Postal Code'].astype(str):
        zip_code = zip_code.replace('-','')[:-2]
        if len(zip_code) < 5:
            correct_zip_code.append(zip_code.zfill(5))
        elif len(zip_code) == 5:
            correct_zip_code.append(zip_code)
        elif len(zip_code) < 9:
            correct_zip_code.append(zip_code.zfill(9)[:5])
        else :
            correct_zip_code.append(zip_code[:5])
    chunk['Provider Business Practice Location Address Postal Code']=correct_zip_code
    # change the column names to lower cases and replace space with underscore
    chunk.columns = [x.lower().replace(' ', '_') for x in chunk.columns]
    # write table to db
    chunk.to_sql('profile', db, if_exists = 'append', index = False)
db.close()

### Write CBSA table to sqlite3 db

In [31]:
%%time
# write zip_cbsa table to db
with sqlite3.connect('data/hopteam.sqlite') as db:
    chunk = pd.read_excel('data/NPPES_Data_Dissemination_February_2022/ZIP_CBSA_122021.xlsx')
    # select and rename 4 columns
    chunk = chunk.loc[:,zip_cbsa_cols]
    chunk = chunk.rename(columns={'usps_zip_pref_city':'city', 'usps_zip_pref_state':'state'})
    # zfill zip to 5 digits
    correct_zip_code = []
    for zip_code in chunk['zip'].astype(str):
        if len(zip_code) <5:
            correct_zip_code.append(zip_code.zfill(5))
        else:
            correct_zip_code.append(zip_code)
    chunk['zip'] = correct_zip_code
    chunk.to_sql('zip_cbsa', db, if_exists = 'append', index = False)    

CPU times: user 5.37 s, sys: 17.6 ms, total: 5.39 s
Wall time: 5.4 s


### Write Taxonomy table to sqlite3 db

In [7]:
# write taxonomy table to db
with sqlite3.connect('data/hopteam.sqlite') as db:
    for chunk in tqdm(pd.read_csv('data/NPPES_Data_Dissemination_February_2022/nucc_taxonomy_220.csv', chunksize = 10000)):
        # select Code and Classification columns
        chunk = chunk.loc[:,taxonomy_cols]
        # change the columns to lower case
        chunk.columns = [x.lower() for x in chunk.columns]
        # write the table to db
        chunk.to_sql('taxonomy', db, if_exists = 'append', index = False)
    

0it [00:00, ?it/s]

In [11]:
db = sqlite3.connect('data/hopteam.sqlite')
db.execute('CREATE INDEX from_referral_id ON referral_from(referral_id)')
db.execute('CREATE INDEX from_npi ON referral_from(from_npi)')
db.execute('CREATE INDEX to_referral_id ON referral_to(referral_id)')
db.execute('CREATE INDEX to_npi ON referral_to(to_npi)')
db.execute('CREATE INDEX referrals_referral_id ON referrals(referral_id)')
db.execute('CREATE INDEX transaction_count ON referrals(transaction_count)')
db.execute('CREATE INDEX average_day_wait ON referrals(average_day_wait)')
db.execute('CREATE INDEX npi ON profile(npi)')
db.execute('CREATE INDEX taxonomy_code ON profile(taxonomy_code)')
db.execute('CREATE INDEX provider_business_practice_location_address_postal_code ON profile(provider_business_practice_location_address_postal_code)')
db.execute('CREATE INDEX zip ON zip_cbsa(zip)')
db.execute('CREATE INDEX code ON taxonomy(code)')
db.close()

### Show tables in sqlite db

In [43]:
# display tables in a sqlite db
query = """
SELECT name FROM sqlite_schema
WHERE type = 'table' AND name NOT LIKE 'sqlite_%';
"""
with sqlite3.connect('data/hopteam.sqlite') as db:
    ref_sqlite = pd.read_sql(query, db)
ref_sqlite

,name
0,referral_from
1,referral_to
2,referrals
3,taxonomy
4,zip_cbsa


In [42]:
# drop a table from sqlite db
db = sqlite3.connect('data/hopteam.sqlite')

#Creating a cursor object using the cursor() method
cursor = db.cursor()

#Doping EMPLOYEE table if already exists
cursor.execute("DROP TABLE profile")
print("Table dropped... ")

#Commit your changes in the database
db.commit()

#Closing the connection
db.close()

Table dropped... 
